In [ ]:
import random
import time

# --- Helper Function ---
def slow_print(text, delay=0.02):
    """Print text slowly for dramatic effect."""
    for c in text:
        print(c, end="", flush=True)
        time.sleep(delay)
    print()

# --- Character Classes ---
class Character:
    def __init__(self, name, hp, attack_power, level=1):
        self.name = name
        self.hp = hp
        self.attack_power = attack_power
        self.level = level

    def is_alive(self):
        return self.hp > 0

    def attack(self, other):
        damage = random.randint(self.attack_power // 2, self.attack_power)
        other.hp -= damage
        slow_print(f"{self.name} attacks {other.name} for {damage} damage!")
        if not other.is_alive():
            slow_print(f"💀 {other.name} was defeated!")

# --- Player Class ---
class Player(Character):
    def __init__(self, name):
        super().__init__(name, hp=80, attack_power=10, level=1)
        self.energy = 0
        self.inventory = []
        self.position = (0, 0)  # start at the center

    def heal(self):
        if "Health Potion" in self.inventory:
            heal_amount = random.randint(15, 35)
            self.hp += heal_amount
            slow_print(f"🧪 You used a Health Potion and healed {heal_amount} HP!")
            self.inventory.remove("Health Potion")
        else:
            slow_print("❌ You have no Health Potions to heal with!")

    def add_item(self, item):
        self.inventory.append(item)
        slow_print(f"🎁 You found a {item}!")
        if item == "Iron Sword":
            self.attack_power += 5
            slow_print("⚔️ Your attack power increased by +5!")
        elif item == "Flame Blade":
            self.attack_power += 8
            slow_print("🔥 Your attack power increased by +8!")
        elif item == "Shadow Spear":
            self.attack_power += 12
            slow_print("🌑 Your attack power increased by +12!")
        elif item == "Armor Plate":
            self.hp += 20
            slow_print("🛡️ Your HP increased by +20!")

# --- Battle System ---
def battle(player, enemy):
    slow_print(f"\n⚔️ A Level {enemy.level} {enemy.name} appears! (HP: {enemy.hp})\n")
    while player.is_alive() and enemy.is_alive():
        print(f"\n{player.name}: {player.hp} HP | {enemy.name}: {enemy.hp} HP (Lv {enemy.level})")
        choice = input("(A)ttack | (H)eal | (R)un | (Q)uit: ").lower()

        if choice == "a":
            player.attack(enemy)
            if enemy.is_alive():
                enemy.attack(player)
        elif choice == "h":
            player.heal()
            if enemy.is_alive():
                enemy.attack(player)
        elif choice == "r":
            if random.random() < 0.4:
                slow_print("🏃 You escaped successfully!")
                return True
            else:
                slow_print("❌ Failed to run away!")
                enemy.attack(player)
        elif choice == "q":
            slow_print("\n👋 You ended your adventure.")
            return "quit"
        else:
            slow_print("Invalid choice!")

    if player.is_alive():
        slow_print(f"🎯 You defeated the {enemy.name}!")
        player.energy += 10
        player.level += 1
        player.hp += 10
        slow_print(f"✨ You feel stronger! Level {player.level} reached. (+10 HP)")
        return True
    else:
        slow_print("☠️ You were slain in battle...")
        return False

# --- Room Setup ---
def create_rooms():
    """Creates a 3x3 grid of rooms with unique features."""
    return {
        (0, 0): {"name": "Central Square", "desc": "The heart of the dungeon. Paths lead in all directions."},
        (0, 1): {"name": "Crystal Cavern", "desc": "Blue crystals shimmer softly.", "item": "Health Potion", "enemy": "Goblin"},
        (1, 0): {"name": "Ancient Armory", "desc": "Rusty weapons line the walls.", "item": "Iron Sword", "enemy": "Skeleton"},
        (-1, 0): {"name": "Flame Chamber", "desc": "The walls glow with molten heat.", "item": "Flame Blade", "enemy": "Orc"},
        (0, -1): {"name": "Dark Sanctum", "desc": "Whispers of the dead fill the air.", "item": "Shadow Spear", "enemy": "Wraith"},
        (1, 1): {"name": "Mystic Garden", "desc": "Overgrown vines hide old relics.", "item": "Armor Plate", "enemy": "Forest Spirit"},
        (-1, -1): {"name": "Frozen Crypt", "desc": "Frost covers ancient tombs.", "item": "Health Potion", "enemy": "Ice Ghoul"},
        (1, -1): {"name": "Sunken Library", "desc": "Dusty tomes speak of lost magic.", "item": "Energy Crystal", "enemy": "Dark Mage"},
        (-1, 1): {"name": "Obsidian Hall", "desc": "Black stone reflects your shadow.", "item": "Health Potion", "enemy": "Shadow Beast"},
    }

# --- Exploration System ---
def explore_room(player, rooms):
    room = rooms.get(player.position, None)
    if not room:
        slow_print("You wander into the void... Better go back!")
        return True

    slow_print(f"\n🏰 You enter the {room['name']}")
    slow_print(room["desc"])

    # Find item
    if "item" in room and random.random() < 0.7:
        player.add_item(room["item"])
        del room["item"]

    # Encounter enemy
    if "enemy" in room and random.random() < 0.8:
        level = random.randint(player.level, player.level + 2)
        enemy = Character(room["enemy"], hp=40 + 10 * level, attack_power=8 + 2 * level, level=level)
        result = battle(player, enemy)
        if result == "quit":
            return "quit"
        if not result:
            return False
        del room["enemy"]

    return True

# --- Movement System ---
def move_player(player, direction, rooms):
    x, y = player.position
    if direction == "n":
        y += 1
    elif direction == "s":
        y -= 1
    elif direction == "e":
        x += 1
    elif direction == "w":
        x -= 1
    else:
        slow_print("Invalid direction! Use N, S, E, or W.")
        return True

    if (x, y) in rooms:
        player.position = (x, y)
        return explore_room(player, rooms)
    else:
        slow_print("🚫 You hit a wall... You can’t go that way.")
        return True

# --- Final Boss ---
def final_boss(player):
    slow_print("\n🔥 The dungeon trembles... The Dark Lord ZALTHOR appears! 🔥")
    boss = Character("Zalthor the Dark Lord", hp=250, attack_power=25, level=12)
    result = battle(player, boss)
    if result == "quit":
        return "quit"
    elif result and player.is_alive():
        slow_print(f"\n🏆 {player.name}, you have defeated Zalthor and saved the realm!")
        return True
    else:
        slow_print("\n💀 Zalthor’s darkness consumes the land...")
        return False

# --- Play Game ---
def play_game():
    slow_print("🌟 WELCOME TO THE REALM OF SHADOWS 🌟")
    name = input("Enter your hero’s name: ")
    player = Player(name)
    rooms = create_rooms()

    slow_print(f"\nWelcome, {player.name}! Explore the dungeon, find treasures, and prepare for the final battle!\n")
    slow_print("Use (N)orth, (S)outh, (E)ast, (W)est to move. Type (Q) to quit.\n")

    while player.is_alive():
        command = input("Move (N/S/E/W) or (Q)uit: ").lower()
        if command == "q":
            slow_print("\n👋 You ended your adventure.")
            return
        result = move_player(player, command, rooms)
        if result == "quit":
            return
        if not result or not player.is_alive():
            slow_print("\n💀 You have fallen in the dungeon...")
            break

        # Victory condition
        remaining_enemies = any("enemy" in r for r in rooms.values())
        if not remaining_enemies:
            slow_print("\n🔥 You’ve cleared all rooms! The Dark Lord awaits in the final chamber...")
            final_boss(player)
            break

    slow_print("\n🎮 Game over!")

# --- Restart Option ---
def restart_menu():
    while True:
        again = input("\nDo you want to play again? (Y/N): ").lower()
        if again == "y":
            main_menu()
            break
        elif again == "n":
            slow_print("\n👋 Thanks for playing! See you next time.")
            break
        else:
            print("Please enter Y or N.")

# --- Main Menu ---
def main_menu():
    while True:
        print("\n==============================")
        print("   ⚔️  REALM OF SHADOWS RPG ⚔️")
        print("==============================")
        print("1. Start New Game")
        print("2. Quit Game")
        choice = input("\nEnter your choice: ")

        if choice == "1":
            play_game()
            restart_menu()
            break
        elif choice == "2":
            slow_print("\n👋 Farewell, brave hero.")
            break
        else:
            print("Invalid choice. Please select 1 or 2.")

# --- Run Game ---
main_menu()



   ⚔️  REALM OF SHADOWS RPG ⚔️
1. Start New Game
2. Quit Game



Enter your choice:  1


🌟 WELCOME TO THE REALM OF SHADOWS 🌟


Enter your hero’s name:  kutuekut



Welcome, kutuekut! Explore the dungeon, find treasures, and prepare for the final battle!

se (N)orth, (S)outh, (E)ast, (W)est to move. Type (Q) to quit.

